In [ ]:
import numpy as np
import pandas as pd
import duckdb
import sqlalchemy
from pandas_gbq import to_gbq

In [28]:
# Connect to DuckDB
con = duckdb.connect("duckdb/br_temp.db")

# Load each table into a separate DataFrame
dim_customers = con.execute("SELECT * FROM olist.dim_customers").fetchdf()
dim_payments = con.execute("SELECT * FROM olist.dim_payments").fetchdf()
dim_reviews = con.execute("SELECT * FROM olist.dim_reviews").fetchdf()
dim_geolocation = con.execute("SELECT * FROM olist.dim_geolocation").fetchdf()
dim_items = con.execute("SELECT * FROM olist.dim_items").fetchdf()
dim_date = con.execute("SELECT * FROM olist.dim_date").fetchdf()
fct_orders = con.execute("SELECT * FROM olist.fct_orders").fetchdf()


In [29]:
def check_one_to_one_mapping(df, key_col, value_cols):
    combo_counts = df.groupby(key_col)[value_cols].nunique()

    # Keep rows where there's more than 1 unique combinations for any value_col
    inconsistent_keys = combo_counts[(combo_counts > 1).any(axis=1)]

    if not inconsistent_keys.empty:
        print(f" {key_col} has non-unique combinations of {value_cols} :")
        return df[df[key_col].isin(inconsistent_keys.index)].sort_values(key_col)
    else:
        print(f"Each '{key_col}' maps uniquely to combinations of {value_cols}.")


In [30]:
check_one_to_one_mapping(dim_geolocation,"geolocation_zip_code_prefix", ["geolocation_lat", "geolocation_lng"] )

 geolocation_zip_code_prefix has non-unique combinations of ['geolocation_lat', 'geolocation_lng'] :


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
278917,01001,-23.549779,-46.633957,sao paulo,SP
93514,01001,-23.549292,-46.633559,sao paulo,SP
647684,01001,-23.550642,-46.634410,sao paulo,SP
186040,01001,-23.549779,-46.633957,são paulo,SP
463454,01001,-23.551337,-46.634027,sao paulo,SP
...,...,...,...,...,...
483660,99980,-28.390507,-51.847413,david canabarro,RS
21871,99980,-28.386408,-51.844876,david canabarro,RS
391715,99980,-28.386239,-51.847741,david canabarro,RS
21867,99990,-28.329472,-51.769109,muliterno,RS


It appears that one zip code can have different combinations of longtitude latittude, source states that the zip code is the first 5 digits.

Therefore, mean of longtitude and latitude will be used as variables for heatmaps for this dataset

<div class="alert alert-block alert-info">
Forming tables for charts

In [31]:
#get buyer city + orders in df
orders_customers = pd.merge(
    fct_orders,
    dim_customers,
    how='left',
    left_on='customer_sid',
    right_on='customer_sid'
)

#if prices are needed
orders_full = pd.merge(
    orders_customers,
    dim_items[['item_sid', 'price']],
    how='left',
    left_on='item_sid',
    right_on='item_sid'
)

In [32]:
orders_full.columns

Index(['payment_sid', 'review_sid', 'item_sid', 'customer_sid', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'TOTAL_PAYMENT', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'load_date', 'price'],
      dtype='object')

In [33]:
orders_customers['order_purchase_timestamp'] = pd.to_datetime(orders_customers['order_purchase_timestamp'])
orders_customers['year_month'] = orders_full['order_purchase_timestamp'].dt.to_period('M')

In [34]:
monthly_city_sales = orders_customers.groupby(['customer_city', 'year_month'])['TOTAL_PAYMENT'].sum().reset_index()

In [35]:
monthly_city_sales['year_month'].value_counts().sort_index(ascending=False)

year_month
2018-10       3
2018-09      13
2018-08    1270
2018-07    1262
2018-06    1257
2018-05    1337
2018-04    1326
2018-03    1383
2018-02    1325
2018-01    1364
2017-12    1201
2017-11    1421
2017-10    1111
2017-09    1012
2017-08    1006
2017-07     973
2017-06     817
2017-05     940
2017-04     696
2017-03     772
2017-02     576
2017-01     381
2016-12       1
2016-10     183
2016-09       4
Freq: M, Name: count, dtype: int64

In [36]:
monthly_city_sales[monthly_city_sales['year_month'] == "2018-09"]

,customer_city,year_month,TOTAL_PAYMENT
1954,barra do pirai,2018-09,55.00
2290,belo horizonte,2018-09,729.82
3678,campina verde,2018-09,46.20
9052,itaguai,2018-09,432.32
10426,joinville,2018-09,52.25
11993,maua,2018-09,390.77
13018,nova friburgo,2018-09,84.20
14967,pirai,2018-09,548.45
17621,santo andre,2018-09,236.80
17807,santos,2018-09,94.40


Comments:

It is interesting to see very few records for sales on 2018-09, 2018-10. It might be beneficial to verify data collection methods during these 2 months.

data frame for top 5 cities (monthly sales) 

In [37]:
city_no_filter = 5
top_cities = (monthly_city_sales.groupby('customer_city')['TOTAL_PAYMENT'].sum().sort_values(ascending=False).head(city_no_filter))
top_cities = top_cities.index.tolist()
top_cities

['sao paulo', 'rio de janeiro', 'belo horizonte', 'brasilia', 'curitiba']

In [38]:
top_city_sales = monthly_city_sales[monthly_city_sales['customer_city'].isin(top_cities)]

In [39]:
#shift customer_city to columns
pivot_df = top_city_sales.pivot(index='year_month', columns='customer_city', values='TOTAL_PAYMENT')
pivot_df.head()

#df for top 5 monthly sales trend chart

customer_city,belo horizonte,brasilia,curitiba,rio de janeiro,sao paulo
year_month,,,,,
2016-09,NaN,NaN,NaN,NaN,28.75
2016-10,1238.20,2360.34,1389.73,3019.85,6484.75
2017-01,2373.44,2162.21,1099.71,11930.12,17999.69
2017-02,5568.46,4602.81,4434.23,24309.30,36128.62
2017-03,12219.22,6139.84,6124.72,33941.80,57865.96


<div class="alert alert-block alert-info">
Daily sales between 2017-10 and 2018-01

In [40]:

fct_orders['order_purchase_timestamp'] = pd.to_datetime(fct_orders['order_purchase_timestamp'])

# Filter to delivered orders and desired date range
mask = (
    (fct_orders['order_status'] == 'delivered') &
    (fct_orders['order_purchase_timestamp'] >= '2017-10-01') &
    (fct_orders['order_purchase_timestamp'] <= '2018-01-31')
)
daily_sales = fct_orders[mask].groupby('order_purchase_timestamp')['TOTAL_PAYMENT'].sum().reset_index()
daily_sales.head()


,order_purchase_timestamp,TOTAL_PAYMENT
0,2017-10-01 00:03:33,42.78
1,2017-10-01 00:06:09,203.14
2,2017-10-01 00:15:12,61.01
3,2017-10-01 00:19:04,52.78
4,2017-10-01 01:10:18,525.09


<div class="alert alert-block alert-info">
Geomap dataframe

In [41]:
#Recall that there are multiple latitude longtitude per zip code. So mean of longtitude latitude is used instead

zip_centroids = dim_geolocation.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean'
}).reset_index()

#filter only delivered orders
delivered_orders = fct_orders[fct_orders['order_status'] == 'delivered']
#merge to get customer zip_code
orders_with_zip = pd.merge(delivered_orders, dim_customers[['customer_sid', 'customer_zip_code_prefix']], how='left',on='customer_sid')

# Merge with centroids 
orders_with_geo = pd.merge(
    orders_with_zip,
    zip_centroids,
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix'
)
geo_sales_heatmap = orders_with_geo.groupby(['geolocation_lat', 'geolocation_lng'])['TOTAL_PAYMENT'].sum().reset_index()
geo_sales_heatmap.head()

#heatmap df total sales 

,geolocation_lat,geolocation_lng,TOTAL_PAYMENT
0,-33.689890,-53.458838,908.16
1,-33.520250,-53.356689,2049.67
2,-32.561073,-53.378161,1242.01
3,-32.235602,-53.078475,457.84
4,-32.193451,-52.179281,35.09


In [42]:
#filters for valid coordinates within brazil

geo_sales_heatmap_filtered = geo_sales_heatmap[
    (geo_sales_heatmap['geolocation_lng'] >= -74) & (geo_sales_heatmap['geolocation_lng'] <= -34) &
    (geo_sales_heatmap['geolocation_lat'] >= -34) & (geo_sales_heatmap['geolocation_lat'] <= 5)].copy()

brazil's longtitude range is -74 to -34

Latitude: from -34 to +5

setting limits for longtitude and latitude

In [43]:

# Plot major cities for heatmap geo total sales
brazil_cities = {
    'São Paulo': (-46.6333, -23.5505),
    'Rio de Janeiro': (-43.1729, -22.9068),
    'Brasília': (-47.9292, -15.7801),
    'Salvador': (-38.5011, -12.9714),
    'Fortaleza': (-38.5433, -3.7172),
    'Manaus': (-60.0258, -3.1019),
    'Curitiba': (-49.2731, -25.4284),
    'Recife': (-34.8770, -8.0476),
    'Porto Alegre': (-51.2300, -30.0346),
    'Belo Horizonte': (-43.9345, -19.9167)
}

<div class="alert alert-block alert-info">
top 10 product category by profit

profit is a derived column. (feature engineering)

In [44]:
orders_full['profit'] = orders_full['TOTAL_PAYMENT'] - orders_full['price']
orders_full.columns

Index(['payment_sid', 'review_sid', 'item_sid', 'customer_sid', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'TOTAL_PAYMENT', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'load_date', 'price', 'profit'],
      dtype='object')

In [45]:
orders_full = orders_full.merge(
    dim_items[['item_sid', 'product_category_name_english']],
    on='item_sid',
    how='left'
)
# Group by category and sum profit
category_profit = orders_full.groupby('product_category_name_english')['profit'].sum().reset_index()

#top 10 ctageories
top10_cat_profit = category_profit.sort_values('profit', ascending=False).head(10)

In [46]:
top10_cat_profit.head()

,product_category_name_english,profit
15,computers_accessories,3050509.22
39,furniture_decor,2442948.47
7,bed_bath_table,1856596.42
57,office_furniture,1714557.72
49,housewares,1664783.85
